## In-Context Learning

`In-context learning (ICL)` enables large language models (LLMs) to perform tasks and generate responses based on the context provided at inference within the input prompt. In practice, the context comprises **one or a few demonstrations of an intended task that condition the model** and allow it to adapt to various tasks without requiring explicit fine-tuning or retraining. This notebook explores the concept of in-context learning, demonstrating how to enable an LLM to implement `Named Entity Recognition (NER)`.

NER is a `Natural Language Processing` task that identifies and classifies named entities (NE) into predefined semantic categories (such as persons, organizations, locations, events, time expressions, and quantities). By converting raw text into structured information, NER makes data more actionable, facilitating tasks like information extraction, data aggregation, analytics, and social media monitoring.

<p align="center">
  <img src="./static/in_context_learning.png">
</p>

### 1. Setup

#### 1.1 Install/Upgrade Python packages

In [ ]:
%pip install openai tenacity --quiet | tail -n 1

#### 1.2 Load packages and OPENAI_API_KEY

You can generate an API key in the OpenAI web interface. See https://platform.openai.com/account/api-keys for details.

This notebook works with the latest OpeanAI models `gpt-4o` and `gpt-4o-mini`.

In [3]:
import logging
import os
import openai

from tenacity import retry, wait_random_exponential, stop_after_attempt

logging.basicConfig(level=logging.INFO, format=' %(asctime)s - %(levelname)s - %(message)s')

OPENAI_MODEL = 'gpt-4o-mini'
client = openai.OpenAI(api_key=os.environ.get("OPENAI_API_KEY", "<your OpenAI API key if not set as env var>"))

### 2. Define the NER labels to be Identified

We define a standard set of NER labels to showcase a wide range of use cases. However, for our specific task of enriching text with knowledge base links, only a subset is practically required.

In [4]:
labels = [
    "person",      # people, including fictional characters
    "fac",         # buildings, airports, highways, bridges
    "org",         # organizations, companies, agencies, institutions
    "gpe",         # geopolitical entities like countries, cities, states
    "loc",         # non-gpe locations
    "product",     # vehicles, foods, appareal, appliances, software, toys 
    "event",       # named sports, scientific milestones, historical events
    "work_of_art", # titles of books, songs, movies
    "law",         # named laws, acts, or legislations
    "language",    # any named language
    "date",        # absolute or relative dates or periods
    "time",        # time units smaller than a day
    "percent",     # percentage (e.g., "twenty percent", "18%")
    "money",       # monetary values, including unit
    "quantity",    # measurements, e.g., weight or distance
]

### 3. Prepare messages

The [chat completions API](https://platform.openai.com/docs/guides/gpt/chat-completions-api) takes a list of messages as input and delivers a model-generated message as an output. While the chat format is primarily designed for facilitating multi-turn conversations, it is equally efficient for single-turn tasks without any preceding conversation. For our purposes, we will specify a message for the system, assistant, and user roles.

#### 3.1 System Message

The `system message` (prompt) sets the assistant's behavior by defining its desired persona and task. We also delineate the specific set of entity labels we aim to identify.

Although one can instruct the model to format its response, it has to be noted that both `gpt-4o` and `gpt-4o-mini` have been fine-tuned to discern when a function should be invoked, and to reply with `JSON` formatted according to the function's signature. This capability streamlines our prompt and enables us to receive structured data directly from the model.

In [5]:
def system_message(labels):
    return f"""
You are an expert in Natural Language Processing. Your task is to identify common Named Entities (NER) in a given text.
The possible common Named Entities (NER) types are exclusively: ({", ".join(labels)})."""

#### 3.2 Assistant Message

`Assistant messages` usually store previous assistant responses. However, as in our scenario, they can also be crafted to provide examples of the desired behavior. While OpenAI is able to execute `zero-shot` Named Entity Recognition, we have found that a `one-shot` approach produces more precise results.

In [6]:
def assisstant_message():
    return f"""
EXAMPLE:
    Text: 'In Germany, in 1440, goldsmith Johannes Gutenberg invented the movable-type printing press. His work led to an information revolution and the unprecedented mass-spread / 
    of literature throughout Europe. Modelled on the design of the existing screw presses, a single Renaissance movable-type printing press could produce up to 3,600 pages per workday.'
    {{
        "gpe": ["Germany", "Europe"],
        "date": ["1440"],
        "person": ["Johannes Gutenberg"],
        "product": ["movable-type printing press"],
        "event": ["Renaissance"],
        "quantity": ["3,600 pages"],
        "time": ["workday"]
    }}
--"""

#### 3.3 User Message

The `user message` provides the specific text for the assistant task:

In [7]:
def user_message(text):
    return f"""
TASK:
    Text: {text}
"""

### 4. Inference

#### 4.1 Chat Completion

The Chat Completions API accepts inputs via the messages parameter, which is an array of message objects:

In [8]:
@retry(wait=wait_random_exponential(min=1, max=10), stop=stop_after_attempt(3))
def run_openai_task(labels, text):
    messages = [
          {"role": "system", "content": system_message(labels=labels)},
          {"role": "assistant", "content": assisstant_message()},
          {"role": "user", "content": user_message(text=text)}
      ]

    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0,
        frequency_penalty=0,
        presence_penalty=0,

    )

    response_message = response.choices[0].message
    
    return {"response": response, 
            "response_message": response_message.content}

#### 4.2 Run OpenAI Task

In [9]:
text = """The Beatles were an English rock band formed in Liverpool in 1960, comprising John Lennon, Paul McCartney, George Harrison, and Ringo Starr.
          Rooted in skiffle, beat and 1950s rock 'n' roll, their sound incorporated elements of classical music and traditional pop in innovative ways.
          The Beatles are the best-selling music act of all time, with estimated sales of 600 million units worldwide.
       """
result = run_openai_task(labels, text)

 2024-08-15 21:11:43,558 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [10]:
logging.info(f"Response: {result['response_message']}")

 2024-08-15 21:11:44,872 - INFO - Response: {
    "org": ["The Beatles"],
    "gpe": ["England", "Liverpool"],
    "date": ["1960"],
    "person": ["John Lennon", "Paul McCartney", "George Harrison", "Ringo Starr"],
    "product": ["rock band"],
    "quantity": ["600 million units"],
    "event": ["1950s rock 'n' roll"]
}
